---
title: "Moving price graph"
format:
  html:
    code-fold: true
jupyter: python3
---

In [ ]:
import os
os.chdir("..")

In [ ]:
from src.jp_imports.data_process import DataTrade
from src.jp_imports.data_pull import DataPull
from polars.testing import assert_frame_equal
import polars as pl
import ibis

ibis.options.interactive = True

d = DataPull("sqlite:///test.sqlite", debug=True, dev=True)
d1 = DataTrade("sqlite:///test.sqlite", debug=True, dev=True)
conn = ibis.sqlite.connect("test.sqlite")

In [ ]:
d.insert_int_jp("test/test_inserts/jp_data_sample.parquet", "data/external/code_agr.json")
d.insert_int_org("test/test_inserts/org_data_sample.parquet")

In [ ]:
df = conn.table("inttradedata")
hts = conn.table("htstable").select("id", "agri_prod").rename(agr_id="id")
df = df.join(hts, df.hts_id == hts.agr_id)
df = df.filter(df.agri_prod)
df.mutate(unit2_id=df.unit2_id.fillna(df.unit1_id))

In [ ]:
time = "monthly"
types = "hts"
df = d1.process_int_org(time, types, agr=False)
df.to_polars()

In [ ]:
for time in ["yearly", "fiscal", "qrt", "monthly"]:
  for types in ["total", "hts", "naics"]:
    for ag in [True, False]:
      d1.process_int_jp(time, types, ag).to_parquet(f"test/test_inserts/jp_results_{time}_{types}_{ag}.parquet")

In [ ]:
df = d1.conn.table("inttradedata")
units = d1.conn.table("unittable")
hts = d1.conn.table("htstable").select("id", "agri_prod").rename(agr_id="id")
df = df.join(hts, df.hts_id == hts.agr_id)
df = df.filter(df.agri_prod).drop(["agr_id", "agri_prod"])
df = df.join(units, df.unit1_id == units.id).rename(unit_1="unit_code")
df = df.mutate(unit2_id=df.unit2_id.fill_null(df.unit1_id))
df = df.join(units, df.unit2_id == units.id).rename(unit_2="unit_code")
df = d1.conversion(df, units)
d1.filter_data(df, ["year", "month", "hts_id"])